In [1]:
import json
import os
import numpy as np

In [66]:
def generate_single_peaked_preference(num, peak_id = None):
    """
        input
            num: number of agent and items
            peak_id: numpy.array of size num
                must start from 0
        return
            rank_list of size num x num 
             
            

    """
    if peak_id is None:
        peak_id =np.random.choice(num,num,replace=True)
    # sort_id = np.argsort(peak_id)
    peak_id = np.expand_dims(peak_id,1)+1
    pref = np.repeat(np.expand_dims(np.linspace(1,num,num,dtype=int),0),num,axis=0)
    pref = num-np.abs(pref - peak_id)
    # break the ties
    pref = (np.argsort(pref,axis=-1)+1)[:,::-1]
    
    return pref


def correct_idx(current_match,matches):
    store_match = current_match
    for match in matches[-1::-1]:
        if match[0] <= current_match[0]:
            current_match = current_match[0]+1,current_match[1]
        if match[1] <= current_match[1]:
            current_match = current_match[0],current_match[1]+1
    matches.append(store_match)
    return current_match, matches

def crawler_step(preference,matches):
    num_agents,num_items = preference.shape
    if num_agents == 0:
        return None
    # (1) if ot ≻it ot+1 holds for some t, let t∗ be the minimal such t
    # this means argmin_t(preference[t,t] > preference[t,t+1])
    
    # preference[t,t] > preference[t,t+1]
    screening = (preference.diagonal(offset=1)-preference.diagonal()[:-1])<0 # agent who (c1) does not want to move to larger one
    # check if (1) is possible
    if screening.sum() == 0:
        t_star = num_agents-1 # if no such agent (c1)
    else:
        t_star = np.argmax(screening) # if there is such agent (c1)
    agent_t_start_choice = np.argmax(preference[t_star]) # the agent's best choice
    next_preference = np.delete(preference, t_star, 0) # remove agent from the preference list
    next_preference = np.delete(next_preference, agent_t_start_choice, 1) # remove item from the preference list
    (t_star,agent_t_start_choice), matches = correct_idx((t_star,agent_t_start_choice),matches=matches) # correct index since the preference list is smaller
    return next_preference,t_star,agent_t_start_choice, matches

def crawler_algorithm(preference):
    """
        preference: a single-peaked preference list where the higher value means more prefered
    """
    matches = []
    result = crawler_step(preference,matches)
    preferences = [preference]
    preference = result[0]
    result_list = [result[1:3]]
    while result is not None:
        result = crawler_step(preference,matches)
        if result is not None:
            preference = result[0]
            preferences.append(result[0])
            matches = result[-1]
            result_list.append(result[1:3])
    return result_list, preferences

def rank_to_score(rank_list):
    """
        rank_list: the value represent the ranking in the preference list
        output:
            preference where the higher value means more prefered
    """
    return len(rank_list) - np.argsort(np.array(rank_list),axis=-1)

def check_single_peaked(preferences):
    """
        preference: a single-peaked preference list where the higher value means more prefered
    """
    peak_id =np.argmax(preferences,axis=-1)
    num = len(preferences)
    # single_peaked = True
    for i,pid in enumerate(peak_id):
        if preferences[i][:pid].any(): # if there are elements on the left
            # unless every elements are less than the elements on the right
            if ((preferences[i][1:pid+1]-preferences[i][:pid])<=0).any():
                return False
        if preferences[i][pid+1:].any(): # if there are elements on the right
            # unless every elements are less than the elements on the left
            if ((preferences[i][pid:-1]-preferences[i][pid+1:])<=0).any():
                return False
    return True

def verify_example_1():
    peak_id = np.array([7,2,7,6,7,3,5])-1
    num = len(peak_id)
    preference = np.array([[7, 6, 5, 4, 3, 2, 1],
                            [2, 3, 1, 4, 5, 6, 7],
                            [7, 6, 5, 4, 3, 2, 1],
                            [5, 7, 6, 4, 3, 2, 1],
                            [7, 6, 5, 4, 3, 2, 1],
                            [3, 4, 2, 5, 1, 6, 7],
                            [5, 6, 4, 7, 3, 2, 1],])
    print(preference)
    preference = rank_to_score(preference)
    # _, pref_idx = sort_preference_list(preference)
    result_list ,_= crawler_algorithm(preference)
    print(np.array(result_list)+1)

def example_1_the_crawler():
    """
        this example is a rank list of all objects, and the corresponding preference values
    """
    example_rank = np.array([
        [4,3,2,1],
        [2,1,3,4],
        [1,2,3,4],
        [2,1,3,4]
    ])
    example_values = np.array([
        [1,2,3,4],
        [3,4,2,1],
        [4,3,2,1],
        [3,4,2,1]
    ])
    return example_rank, example_values


In [64]:
# check single-peak
print(check_single_peaked(np.array([[1,3,3,4]])))
print(check_single_peaked(example_1_the_crawler()[1]))
print(check_single_peaked(rank_to_score(generate_single_peaked_preference(100))))


False
True
True


In [3]:

import plotly.graph_objects as go
import ipywidgets as widgets
import pandas as pd

In [60]:
# example 1 from "The crawler"
preference = rank_to_score(example_1_the_crawler()[0])
x = np.arange(1,len(preference)+1)
result_list, preferneces = crawler_algorithm(preference)
print(result_list)
num = len(preference)
fig = go.Figure()
for i,pref in enumerate(preference):
    fig.add_trace(go.Scatter(x=x, y=pref,
                        mode='lines+markers',
                        name=f'agent {i+1}'))
for i,pref in enumerate(preference):
    result = result_list[i]
    fig.add_trace(go.Scatter(x=[result[1]+1], y=[preference[result[0]][result[1]]],
                        mode='markers',
                        marker=dict(
                            color='LightSkyBlue',
                            size=20,
                            line=dict(
                                color='MediumPurple',
                                width=2
                            )
                        ),
                        hovertext=f'Match agent {result[0]+1} with object {result[1]+1}',
                        hoverinfo='text+name',
                        name=f'match {i+1}'))
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    ),
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    ),
)

int_range = widgets.IntSlider(value=0,max=num,min=0)

display(fig)

[(1, 1), (2, 0), (3, 2), (0, 3)]


In [61]:
# example 1 from
peak_id = np.array([7,2,7,6,7,3,5])-1
num = len(peak_id)

preference = generate_single_peaked_preference(num,peak_id)
result_list, preferneces = crawler_algorithm(preference)
agent_left = [[i for i in range(1,num) if i not in np.array(result_list)[:,0]] for r in range(len(result_list))]

fig = go.FigureWidget(go.Figure(data=[go.Table(header=dict(values=list(range(1,8))),
                 cells=dict(values=preference.T,fill={'color':'#E8EDDF'}))
                     ]))
i = 0

int_range = widgets.IntSlider(value=0,max=num,min=0)
output2 = widgets.Output()

# display(int_range, output2)
def map_color(i):
    if i ==0:
        color_change_cells = np.array([])
    else:
        color_change_cells = np.array(result_list[:i])
    # print(color_change_cells)
    fill_color = []
    for row in range(num):
        row_colors = []
        for col in range(num):
            if len(color_change_cells) == 0:
                row_colors.append('#E8EDDF')
            else:
                if [col,row] in color_change_cells.tolist():
                    row_colors.append('#F5CB5C')
                elif col in color_change_cells[:,0] or row in color_change_cells[:,1]:
                    row_colors.append('#CFDBD5')
                else: # base
                    row_colors.append('#E8EDDF')
        fill_color.append(row_colors)
    # print(fill_color)
    return fill_color
def on_value_change(change):
    fill_color = map_color(change['new'])
    fig.update_traces( {'cells': {'fill': {'color': fill_color}},})
    with output2:
        print(result_list[:change['new']])

int_range.observe(on_value_change, names='value')
visual_tab  = widgets.VBox([
    int_range,fig])
display(visual_tab)

    'data': [{'cells': {'fill': {'color': '#E8EDDF'},…

In [67]:
verify_example_1()

[[7 6 5 4 3 2 1]
 [2 3 1 4 5 6 7]
 [7 6 5 4 3 2 1]
 [5 7 6 4 3 2 1]
 [7 6 5 4 3 2 1]
 [3 4 2 5 1 6 7]
 [5 6 4 7 3 2 1]]
[[2 2]
 [6 3]
 [4 5]
 [7 6]
 [5 7]
 [3 4]
 [1 1]]
